# Divisions for a deep network
---

In [1]:
# Testing BatchNorm overhead
from models.recursives import Conv_Net

def count_parameters(net):
    pars = 0
    for _, parm in net.state_dict().items():
        pars += parm.shape.numel()
    return pars

# For small Network
cnn1 = Conv_Net('cnn1', M=12, L=4, normalize=False)
cnn2 = Conv_Net('cnn2', M=12, L=4, normalize=True)
p1 = count_parameters(cnn1); p2 = count_parameters(cnn2)
print('No BN: {} || BN: {}  ||  Ratio: {}'.format(p1, p2, p1/p2))

# For deep Network
cnn1 = Conv_Net('cnn1', M=32, L=16, normalize=False)
cnn2 = Conv_Net('cnn2', M=32, L=16, normalize=True)
p1 = count_parameters(cnn1); p2 = count_parameters(cnn2)
print('No BN: {} || BN: {}  ||  Ratio: {}'.format(p1, p2, p1/p2))


No BN: 15238 || BN: 15336  ||  Ratio: 0.9936098069900887
No BN: 174634 || BN: 174892  ||  Ratio: 0.9985248038789653


The Single Deep NN is defined below

In [2]:
class Net:
    
    def __init__(self,M,L):
        self.M = M
        self.L = L
    
    def V(self):
        return 8*8*3*self.M
    
    def W(self):
        return (3*3*self.M*self.M*self.L) + (self.M*(self.L+1))

    def F(self):
        return (64*self.M*10) + 10
    
    def total(self):
        return self.V() + self.W() + self.F()
    
L = 16
M = 32
S = Net(M = M, L = L)
print('M = {}, L = {}'.format(M, L))
print(S.total())

M = 32, L = 16
174634


### A - Horizontal Division
![](images/A_horizontal.png)


In [28]:
# In Horizontal Division we need to calculate Me given Le = L or a custom Le

# A: Get the value of M keeping L the same as the deep network:
def getM(S,K):
    ensemble_network = Net(M = 1, L = S.L)
    budget = S.total()/K
    if K == 1:
        return S.M
        
    # print("Budget: " + str(budget))
    for M in range(S.M):
        ensemble_network.M = M
        if ensemble_network.total() > budget:
            return M-1

# AB: Get the value of M given an L different from the deep network:
def getM_L(S,K,L):
    ensemble_network = Net(M = 1, L = L)
    budget = S.total()/K

    # sweep M
    for M in range(S.M):
        ensemble_network.M = M
        if ensemble_network.total() == budget:
            return M
        if ensemble_network.total() > budget:
            return M-1
    return -1

In [36]:
## A: HORIZONTAL DIVISION: Fix L, K --> Divide M 
Le = S.L
Me = getM(S, K=4)
Ek = Net(M = Me, L=Le)
print('Me = {}, Le = {}'.format(Me, Le))
print(S.total() / Ek.total())

## AB: CONDITIONED HORIZONTAL DIVISION: Fix K, Choose L --> Divide M
Le = 4
Me = getM_L(S, L=Le, K=4)
Ek = Net(M = Me, L = Le)
print('Me = {}, Le = {}'.format(Me, Le))
print(S.total() / Ek.total())

Me = 14, Le = 16
4.352791625124626
Me = 25, Le = 4
4.020582479567169


### B - Vertical Division
![](images/B_vertical.png)

In [37]:
# In Horizontal Division we need to calculate Le given Me = M or a custom Me

# Get the value of L keeping M the same as the deep network:
def getL(S,K):
    ensemble_network = Net(M = 1, L = S.L)
    budget = S.total()/K
    print("Budget: " + str(budget))
    for L in range(S.L):
        ensemble_network.L = L
        if ensemble_network.total() > budget:
            return L-1
    return L  ## TODO: M=1 is allowing to have Le > L for k=4 and returns None

# Get the value of L keeping given an M different from the deep network:
def getL_M(S,K,M):
    ensemble_network = Net(M = M , L = S.L)
    budget = S.total()/K
    
    for L in range(S.L):
        ensemble_network.L = L
        if ensemble_network.total() == budget:
            return L
        if ensemble_network.total() > budget:
            return L-1
    return -1

In [38]:
## B: VERTICAL DIVISION: Fix M, K --> Divide L
Me = S.M
Le = getL(S, K = 4)
Ek = Net(M = Me, L = Le)
print('Me = {}, Le = {}'.format(Me, Le))
print(S.total() / Ek.total())

## BA: CONDITIONED VERICAL DIVISION: Fix K, Choose M --> Divide L
Me = 16
Le = getL_M(S = S, M = Me, K = 4)
Ek = Net(M = Me, L = Le)
print('Me = {}, Le = {}'.format(Me, Le))     ## Le should be being greater than 15
print(S.total() / Ek.total())

Budget: 43658.5
Me = 32, Le = 15
1.0559176713869372
Me = 16, Le = 13
4.014759299278127


### C - Recursive Division
![](images/C_recursive.png)

In [35]:
## CA: RECURSIVE: Fix Le = 1, Choose M --> Calculate Ensemble Size allowed
Le = 1
Me = S.M
Ek = Net(M = Me, L = Le)
print(S.total() / Ek.total())

## CA: RECURSIVE: Fix Le = 1, Choose K --> Calculate Me allowed to use
K = 6
Le = 1
Me = getM_L(S, L=Le, K=K)
Ek = Net(M = Me, L = Le)
print(S.total() / Ek.total())

4.862560561340981
6.003437725600743
